In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.multioutput import MultiOutputRegressor

data = pd.read_csv('GCPW_Training_mid/GCPW_1_3_mid.csv')
data_int=data.iloc[:,:]

data_int_10=np.tile(data_int, (25, 1)) #串聯10倍長度
mean,var = 0,0.01
noise = np.random.normal(mean, var, data_int_10.shape[0]) #並加入noise
noise = np.random.normal(mean, var, data_int_10.shape) #並加入noise
print(noise.shape)
data_noise_10 = data_int_10+noise*0.00001 #嘗試在不同大小的noise
X = data_noise_10[:, :3]
y = data_noise_10[:, 3:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)



(121500, 10)
(97200, 3) (24300, 3) (97200, 7) (24300, 7)


In [ ]:
xgb = XGBRegressor(objective='reg:squarederror',
                   seed=42)
param_grid = {
    'estimator__learning_rate': [0.01,0.05,0.1,0.2],
    'estimator__n_estimators': [800,900,1000],
    'estimator__max_depth': [8,9,10,11,12],
       'estimator__min_child_weight': [5 , 7],
       'estimator__gamma': [0.0,0.1,0.2],
       'estimator__subsample': [0.8, 0.9 , 1]
}
 
multi_output_model = MultiOutputRegressor(xgb)

grid_search = GridSearchCV(estimator=multi_output_model,
                           param_grid=param_grid,
                           cv=2,
                           scoring='neg_mean_squared_error',
                           verbose=1,
                           n_jobs=-1
                          )

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')

best_xgb_model = XGBRegressor(objective='reg:squarederror',
                              learning_rate=best_params['estimator__learning_rate'],
                              n_estimators=best_params['estimator__n_estimators'],
                              max_depth=best_params['estimator__max_depth'],
                              min_child_weight=best_params['estimator__min_child_weight'],
                              gamma=best_params['estimator__gamma'],
                              subsample=best_params['estimator__subsample'],
                              colsample_bytree=0.8,
                              reg_alpha=0,
                              reg_lambda=1,
                              seed=42)
best_multi_output_model = MultiOutputRegressor(best_xgb_model)

best_multi_output_model.fit(X_train, y_train)

y_pred = best_multi_output_model.predict(X_test)
# 
r2_scores = r2_score(y_test, y_pred, multioutput='raw_values')
print(f'R² scores for each output: {r2_scores}')

Fitting 2 folds for each of 1080 candidates, totalling 2160 fits


In [ ]:
RMSE=np.sqrt(sum((y_test-y_pred) ** 2)/(y_pred.size))
print('RMSE [S11,S21,beta,alpha,Zo] : ')#,S11(angle),S21(angle)
print(RMSE)
NRMSE=RMSE/abs(np.mean(y_pred, axis=0))
print('NRMSE [S11,S21,beta,alpha,Zo] :')# ,S11(angle),S21(angle)
print(NRMSE)

In [ ]:
best_xgb_model = XGBRegressor(
    objective='reg:squarederror',
    learning_rate=best_params['estimator__learning_rate'],
    n_estimators=best_params['estimator__n_estimators'],
    max_depth=best_params['estimator__max_depth'],
     min_child_weight=best_params['estimator__min_child_weight'],
     gamma=best_params['estimator__gamma'],
     subsample=best_params['estimator__subsample'],
    seed=42
)
head=401*0 #後面乘上的數字等於第幾組數據 數據in平板 因為81個頻率點
data = pd.read_csv('GCPW_Training_mid/GCPW_freq_mid.csv')##val_input.csv
X_val = data.iloc[:,:]
# 
best_multi_output_model = MultiOutputRegressor(best_xgb_model)

best_multi_output_model.fit(X_train, y_train)

y_pred_val = best_multi_output_model.predict(X_val)

y_pred_val_df = pd.DataFrame(y_pred_val,columns=['s11','s21','beta','alpha','Zo','S11(angle)','S21(angle)'])
y_pred_val_df.to_csv('GCPW_val/GCPW_pred_1_3_mid.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

k = 0
head_81=k*81
head_401=k*401

data_int = np.array(data_int)
freq_81=np.arange(37, 41.05, 0.05)
freq_401=np.arange(37, 41.01, 0.01)
plt.plot(freq_81, data_int[head_81:head_81+81, 3], label='simulation data')
plt.plot(freq_401, y_pred_val[head_401:head_401+401, 1], label='regression data')
plt.xlabel('Frequency')
plt.ylabel('Value')
plt.title('S21')
plt.legend()
plt.show()

#beta
plt.plot(freq_81, data_int[head_81:head_81+81, 5], label='simulation data')
plt.plot(freq_401, y_pred_val[head_401:head_401+401, 2], label='regression data')
plt.xlabel('Frequency')
plt.ylabel('Value')
plt.title('beta')
plt.legend()
plt.show()

#alpha
plt.plot(freq_81, data_int[head_81:head_81+81, 6], label='simulation data')
plt.plot(freq_401, y_pred_val[head_401:head_401+401, 3], label='regression data')
plt.xlabel('Frequency')
plt.ylabel('Value')
plt.title('alpha')
plt.legend()
plt.show()

#Zo
plt.plot(freq_81, data_int[head_81:head_81+81, 7], label='simulation data')
plt.plot(freq_401, y_pred_val[head_401:head_401+401, 4], label='regression data')
plt.xlabel('Frequency')
plt.ylabel('Value')
plt.title('Zo')
plt.legend()
plt.show()